In [1]:
import cv2
import numpy as np
import os

# ==== SETTINGS ====
video_path = 'data/A/vidd2.mp4'
output_input_dir = 'output/paired/input'
output_target_dir = 'output/paired/target'
os.makedirs(output_input_dir, exist_ok=True)
os.makedirs(output_target_dir, exist_ok=True)

# ==== VIDEO SETUP ====
cap = cv2.VideoCapture(video_path)
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))  # Get total frame count
frame_count = 0
saved_frame_count = 0

# CLAHE setup
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))

while True:
    ret, frame = cap.read()
    if not ret:
        break
    frame_count += 1

    # Skip first 10 frames
    if frame_count <= 10:
        continue

    # === Show Progress ===
    print(f"Processing frame {frame_count}/{total_frames} -> saved pair #{saved_frame_count}", end='\r')

    # Save original frame as "noisy" input
    input_img_path = os.path.join(output_input_dir, f"frame_{saved_frame_count:04d}.png")
    cv2.imwrite(input_img_path, frame)

    # === Image Enhancement Pipeline ===

    ycrcb = cv2.cvtColor(frame, cv2.COLOR_BGR2YCrCb)
    ycrcb[:, :, 0] = cv2.equalizeHist(ycrcb[:, :, 0])
    equalized_img = cv2.cvtColor(ycrcb, cv2.COLOR_YCrCb2BGR)

    ycrcb = cv2.cvtColor(equalized_img, cv2.COLOR_BGR2YCrCb)
    ycrcb[:, :, 0] = clahe.apply(ycrcb[:, :, 0])
    clahe_img = cv2.cvtColor(ycrcb, cv2.COLOR_YCrCb2BGR)

    clahe_img = clahe_img.astype(np.int16)
    clahe_img = np.clip(clahe_img, 0, 255).astype(np.uint8)

    # Save enhanced image as target
    target_img_path = os.path.join(output_target_dir, f"frame_{saved_frame_count:04d}.png")
    cv2.imwrite(target_img_path, clahe_img)

    saved_frame_count += 1

cap.release()
print(f"\n✅ Finished: Saved {saved_frame_count} paired frames to '{output_input_dir}' and '{output_target_dir}'.")



: 

In [5]:
import cv2
import numpy as np
import os

# ==== SETTINGS ====
video_path = 'data/A/vidd.mp4'  # Input video
output_input_dir = 'output/paired/input'   # Save original frames here
output_target_dir = 'output/paired/target'  # Save enhanced frames here

os.makedirs(output_input_dir, exist_ok=True)
os.makedirs(output_target_dir, exist_ok=True)

# ==== VIDEO SETUP ====
cap = cv2.VideoCapture(video_path)
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
frame_count = 0
saved_frame_count = 0

# ==== Enhancer Setup ====
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
denoise_kernel_size = 5  # Set to 0 or 1 to disable
kernel_sharpen = np.array([[0, -1, 0],
                           [-1,  5, -1],
                           [0, -1, 0]])

while True:
    ret, frame = cap.read()
    if not ret:
        break
    frame_count += 1

    # Skip first 10 frames if needed
    if frame_count <= 10:
        continue

    print(f"Processing frame {frame_count}/{total_frames} -> saved pair #{saved_frame_count}", end='\r')

    # ==== Save Original Frame ====
    input_path = os.path.join(output_input_dir, f"frame_{saved_frame_count:04d}.png")
    cv2.imwrite(input_path, frame)

    # ==== Enhancement Pipeline ====
    img_yuv = cv2.cvtColor(frame, cv2.COLOR_BGR2YCrCb)
    y, cr, cb = cv2.split(img_yuv)

    # CLAHE on luminance
    y_clahe = clahe.apply(y)

    # Optional denoising
    y_processed = y_clahe
    if denoise_kernel_size > 1 and denoise_kernel_size % 2 == 1:
        y_denoised = cv2.GaussianBlur(y_clahe, (denoise_kernel_size, denoise_kernel_size), 0)
        y_processed = y_denoised

    # Sharpening
    y_sharp = cv2.filter2D(y_processed, -1, kernel_sharpen)

    # Merge and convert back
    processed_yuv = cv2.merge([y_sharp, cr, cb])
    processed_bgr = cv2.cvtColor(processed_yuv, cv2.COLOR_YCrCb2BGR)

    # Clip & Save Output
    processed_bgr = np.clip(processed_bgr.astype(np.int16), 0, 255).astype(np.uint8)
    output_path = os.path.join(output_target_dir, f"frame_{saved_frame_count:04d}.png")
    cv2.imwrite(output_path, processed_bgr)

    saved_frame_count += 1

cap.release()
print(f"\n✅ Finished: Saved {saved_frame_count} paired frames to:")
print(f"  Input:  '{output_input_dir}'")
print(f"  Target: '{output_target_dir}'")

KeyboardInterrupt: 